In [2]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas_ta as ta
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import datetime

/usr/local/lib/python3.8/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [ ]:
!pip install backtesting
!pip install yfinance 
!pip install pandas_ta

In [60]:
# ticker and the start and end dates for testing
ticker =  input("Enter ticker: ")
start = datetime.datetime(2013, 1, 1)
end = datetime.datetime(2023, 1, 1)

Enter ticker: ^GDAXI


In [61]:
df=yf.download( ticker , start=start,end=end)
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2013-01-02,7689.459961,7789.939941,7689.459961,7778.779785,7778.779785,140345500
2013-01-03,7762.129883,7777.020020,7737.979980,7756.439941,7756.439941,86174100
2013-01-04,7740.990234,7779.669922,7727.549805,7776.370117,7776.370117,101577000
2013-01-07,7760.009766,7767.549805,7713.419922,7732.660156,7732.660156,151695400
2013-01-08,7710.700195,7754.379883,7683.479980,7695.830078,7695.830078,130425500
...,...,...,...,...,...,...
2022-12-23,13945.589844,14000.679688,13874.500000,13940.929688,13940.929688,28738700
2022-12-27,14047.419922,14063.139648,13966.349609,13995.099609,13995.099609,22975000
2022-12-28,14013.719727,14018.469727,13914.620117,13925.599609,13925.599609,27583800


In [62]:
def indicator(df):
    bbands = ta.bbands(close = df.Close.s, std = 1,)
    return bbands.to_numpy().T[0:3]

In [63]:
class BBStrategy(Strategy):
  
  def init(self):
        self.bbands = self.I(indicator,self.data)
    
  def next(self):
      lower_band = self.bbands[0]
      upper_band = self.bbands[2]
      if self.position:
          if self.data.Close[-1] > upper_band[-1]:
                self.position.close()
      else:
          print(self.data.Close[-1], lower_band[-1])
          if self.data.Close[-1] < lower_band[-1]:
                self.buy()

In [64]:
bt=Backtest(df,BBStrategy,cash=100_000,commission=0.001) 
stats=bt.run()
print(stats)

7720.47021484375 7708.379605915365
7708.47021484375 7699.095329207067
7715.52978515625 7702.333402418699
7729.52001953125 7702.599265482038
7675.91015625 7691.618755784998
7702.22998046875 7684.230538262263
7748.85986328125 7683.389254317623
7696.2099609375 7691.752833691112
7707.5400390625 7697.614553031168
7748.1298828125 7697.531385438165
7857.97021484375 7694.56960646731
7833.0 7702.962012260941
7848.56982421875 7738.969805324634
7811.31005859375 7780.6358305951435
7776.0498046875 7796.09044011182
7711.89013671875 7610.515167889552
7631.18994140625 7628.673769101512
7593.509765625 7606.943628873984
7728.89990234375 7604.857497581604
7583.56982421875 7587.429651502887
7597.10986328125 7595.505297782149
7675.830078125 7590.721876364133
7741.7001953125 7627.961166717403
7708.16015625 7638.651186220547
7691.68017578125 7634.765934335016
7870.31005859375 7667.651071800553
7919.330078125 7694.800297244156
7939.77001953125 7720.450445171853
7986.47021484375 7779.529722941473
7984.29003906

In [65]:
bt.plot()

Row(id='5855', ...)

In [59]:
stats['_trades']  # Contains individual trade data

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,15,22,23,6283.176802,6347.200195,960.350896,0.010190,2013-02-01,2013-02-04,3 days
1,16,24,30,6253.046604,6338.399902,1365.652766,0.013650,2013-02-05,2013-02-13,8 days
2,16,34,35,6324.518396,6379.100098,873.307234,0.008630,2013-02-19,2013-02-20,1 days
3,16,37,43,6297.791500,6378.600098,1292.937563,0.012831,2013-02-22,2013-03-04,10 days
4,16,54,62,6464.357802,6411.700195,-842.521711,-0.008146,2013-03-19,2013-04-02,14 days
...,...,...,...,...,...,...,...,...,...,...
187,17,2437,2449,7495.588198,7351.100098,-2456.297702,-0.019276,2022-08-24,2022-09-12,19 days
188,18,2454,2465,7243.936896,7086.500000,-2833.864119,-0.021734,2022-09-20,2022-10-05,15 days
189,18,2469,2474,6966.259104,6920.200195,-829.060365,-0.006612,2022-10-11,2022-10-18,7 days
190,16,2510,2515,7528.921302,7502.899902,-416.342398,-0.003456,2022-12-07,2022-12-14,7 days
